Variational AutoEncoder (VAE) with CelebA
=========================================
---
Formation Introduction au Deep Learning  (FIDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Episode 1 - Train a model

 - Defining a VAE model
 - Build the model
 - Train it
 - Follow the learning process with Tensorboard


## Step 1 - Setup environment
### 1.1 - Python stuff

In [1]:
import numpy as np
import os,sys
from importlib import reload

import modules.vae
import modules.data_generator

reload(modules.data_generator)
reload(modules.vae)

from modules.vae  import VariationalAutoencoder
from modules.data_generator import DataGenerator

sys.path.append('..')
import fidle.pwk as ooo
reload(ooo)

ooo.init()

VariationalAutoencoder.about()
DataGenerator.about()


FIDLE 2020 - Practical Work Module
Version              : 0.2.8
Run time             : Tuesday 11 February 2020, 21:52:21
TensorFlow version   : 2.0.0
Keras version        : 2.2.4-tf

FIDLE 2020 - Variational AutoEncoder (VAE)
TensorFlow version   : 2.0.0
VAE version          : 1.27

FIDLE 2020 - DataGenerator
Version              : 0.4


### 1.2 - The good place

In [2]:
place, dataset_dir = ooo.good_place( { 'GRICAD' : '/bettik/PROJECTS/pr-fidle/datasets/celeba',
                                       'IDRIS'  : f'{os.getenv("WORK","")}/datasets/celeba'    } )

# ---- train/test datasets

train_dir    = f'{dataset_dir}/clusters.train'
test_dir     = f'{dataset_dir}/clusters.test'

Well, we should be at IDRIS !
We are going to use: /gpfswork/rech/mlh/uja62cb/datasets/celeba


## Step 2 - DataGenerator and validation data
Ok, everything's perfect, now let's instantiate our generator for the entire dataset.

In [3]:
data_gen = DataGenerator(train_dir, 32, k_size=1)
x_test   = np.load(f'{test_dir}/images-000.npy')

print(f'Data generator : {len(data_gen)} batchs of {data_gen.batch_size} images, or {data_gen.dataset_size} images')
print(f'x_test : {len(x_test)} images')

Data generator : 6250 batchs of 32 images, or 200000 images
x_test : 2599 images


## Step 3 - Get VAE model

In [4]:
tag = 'CelebA.003'

input_shape = (128, 128, 3)
z_dim       = 200
verbose     = 0

encoder= [ {'type':'Conv2D',          'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2D',          'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
         ]

decoder= [ {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':64, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':32, 'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'relu'},
           {'type':'Dropout',         'rate':0.25},
           {'type':'Conv2DTranspose', 'filters':3,  'kernel_size':(3,3), 'strides':2, 'padding':'same', 'activation':'sigmoid'}
         ]

vae = modules.vae.VariationalAutoencoder(input_shape    = input_shape, 
                                         encoder_layers = encoder, 
                                         decoder_layers = decoder,
                                         z_dim          = z_dim, 
                                         verbose        = verbose,
                                         run_tag        = tag)
vae.save(model=None)

Model initialized.
Outputs will be in  : ./run/CelebA.003
Config saved in     : ./run/CelebA.003/models/vae_config.json


## Step 4 - Compile it

In [5]:
optimizer     = 'adam'
r_loss_factor = 10000

vae.compile(optimizer, r_loss_factor)

Compiled.


## Step 5 - Train
Run time at IDRIS :
 - Initial example, without data optimisation : 1950 sec - 0:32:30
 - This Notebook : 642 sec. - 0:10:42  

In [6]:
epochs            = 10
initial_epoch     = 0

In [7]:
vae.train(data_generator    = data_gen,
          x_test            = x_test,
          epochs            = epochs,
          initial_epoch     = initial_epoch
         )

Epoch 1/10
6250/6250 [==============================] - 134s 21ms/step - loss: 276.7400 - vae_r_loss: 225.1146 - vae_kl_loss: 51.6251 - val_loss: 218.9625 - val_vae_r_loss: 168.7354 - val_vae_kl_loss: 50.3250
Epoch 2/10
6250/6250 [==============================] - 129s 21ms/step - loss: 230.0932 - vae_r_loss: 175.5696 - vae_kl_loss: 54.5233 - val_loss: 211.9046 - val_vae_r_loss: 159.8557 - val_vae_kl_loss: 52.1412
Epoch 3/10
6250/6250 [==============================] - 130s 21ms/step - loss: 224.8574 - vae_r_loss: 169.8087 - vae_kl_loss: 55.0488 - val_loss: 206.6012 - val_vae_r_loss: 152.5130 - val_vae_kl_loss: 54.2148
Epoch 4/10
6250/6250 [==============================] - 130s 21ms/step - loss: 222.4147 - vae_r_loss: 167.3425 - vae_kl_loss: 55.0722 - val_loss: 205.5673 - val_vae_r_loss: 152.6146 - val_vae_kl_loss: 53.0560
Epoch 5/10
6250/6250 [==============================] - 130s 21ms/step - loss: 221.0805 - vae_r_loss: 165.9933 - vae_kl_loss: 55.0873 - val_loss: 203.2717 - val_vae